In [1]:
!pip install ipywidgets

In [9]:
import pandas as pd
import numpy as np
import joblib
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

class TrafficDashboard:
    def __init__(self):
        # Load models and scalers
        print("Loading models and scalers...")
        self.dt_model = joblib.load('decision_tree_model.joblib')
        self.lr_model = joblib.load('linear_regression_model.joblib')
        self.dt_scaler = joblib.load('dt_scaler.joblib')
        self.lr_scaler = joblib.load('lr_scaler.joblib')
        
        # Load historical data for better feature calculation
        self.historical_data = pd.read_csv('engineered_traffic_data.csv')
        
        # Create widgets
        self.create_widgets()
        
    def create_widgets(self):
        # Time inputs
        self.hour_widget = widgets.IntSlider(
            value=12,
            min=0,
            max=23,
            description='Hour:',
            style={'description_width': 'initial'}
        )
        
        self.is_weekend_widget = widgets.Checkbox(
            value=False,
            description='Is Weekend',
            style={'description_width': 'initial'}
        )
        
        # Weather inputs
        self.temperature_widget = widgets.FloatSlider(
            value=70.0,
            min=30.0,
            max=100.0,
            description='Temperature (°F):',
            style={'description_width': 'initial'}
        )
        
        self.humidity_widget = widgets.IntSlider(
            value=50,
            min=0,
            max=100,
            description='Humidity (%):',
            style={'description_width': 'initial'}
        )
        
        self.wind_speed_widget = widgets.FloatSlider(
            value=5.0,
            min=0.0,
            max=30.0,
            description='Wind Speed (mph):',
            style={'description_width': 'initial'}
        )
        
        self.visibility_widget = widgets.IntSlider(
            value=5000,
            min=0,
            max=10000,
            description='Visibility (m):',
            style={'description_width': 'initial'}
        )
        
        self.precipitation_widget = widgets.FloatSlider(
            value=0.0,
            min=0.0,
            max=50.0,
            description='Precipitation (mm):',
            style={'description_width': 'initial'}
        )
        
        self.weather_widget = widgets.Dropdown(
            options=['Clear', 'Clouds', 'Mist', 'Fog', 'Rain', 'Snow', 'Thunderstorm'],
            value='Clear',
            description='Weather:',
            style={'description_width': 'initial'}
        )
        
        # Traffic history inputs (with more reasonable ranges)
        avg_traffic = self.historical_data['traffic_count'].mean()
        self.rolling_avg_widget = widgets.IntSlider(
            value=int(avg_traffic),
            min=0,
            max=2000,
            description='3-Hr Rolling Avg:',
            style={'description_width': 'initial'}
        )
        
        self.traffic_density_widget = widgets.FloatSlider(
            value=0.05,  # Typical value from historical data
            min=0.0,
            max=0.2,
            step=0.01,
            description='Traffic Density:',
            style={'description_width': 'initial'}
        )
        
        # Update button
        self.update_button = widgets.Button(
            description='Update Predictions',
            style={'description_width': 'initial'}
        )
        self.update_button.on_click(self.update_predictions)
        
        # Output widget for plots
        self.output = widgets.Output()
    
    def get_typical_values(self, hour, is_weekend):
        """Get typical traffic values for given hour and day type."""
        mask = (self.historical_data['hour'] == hour) & \
               (self.historical_data['is_weekend'] == is_weekend)
        
        typical_data = self.historical_data[mask]
        
        return {
            'rolling_avg': typical_data['rolling_avg_3h'].median(),
            'traffic_density': typical_data['traffic_density'].median(),
        }
    
    def create_feature_vector(self):
        """Create feature vector from widget values with better scaling."""
        hour = self.hour_widget.value
        is_weekend = self.is_weekend_widget.value
        
        # Get typical values for current conditions
        typical_values = self.get_typical_values(hour, is_weekend)
        
        # Use these for scaling if widget values aren't set
        rolling_avg = self.rolling_avg_widget.value or typical_values['rolling_avg']
        traffic_density = self.traffic_density_widget.value or typical_values['traffic_density']
        
        weather_severity = {
            'Clear': 0, 'Clouds': 1, 'Mist': 2, 'Fog': 3,
            'Rain': 4, 'Snow': 5, 'Thunderstorm': 6
        }
        
        features = {
            'hour_sin': np.sin(2 * np.pi * hour/24),
            'hour_cos': np.cos(2 * np.pi * hour/24),
            'is_morning_peak': 1 if 6 <= hour <= 9 else 0,
            'is_evening_peak': 1 if 16 <= hour <= 19 else 0,
            'is_weekend': int(is_weekend),
            'day_of_week_num': 6 if is_weekend else 2,  # Sample weekday/weekend
            'month': datetime.now().month,
            'temperature': self.temperature_widget.value,
            'humidity': self.humidity_widget.value,
            'wind_speed': self.wind_speed_widget.value,
            'visibility': self.visibility_widget.value,
            'precipitation': self.precipitation_widget.value,
            'weather_severity': weather_severity[self.weather_widget.value],
            'rolling_avg_3h': rolling_avg,
            'traffic_density': traffic_density
        }
        
        return pd.DataFrame([features])
    
    def update_predictions(self, _):
        """Update predictions and plots."""
        with self.output:
            clear_output(wait=True)
            
            # Get feature vector
            features = self.create_feature_vector()
            
            # Make predictions
            X_scaled_dt = self.dt_scaler.transform(features)
            X_scaled_lr = self.lr_scaler.transform(features)
            
            congestion_level = self.dt_model.predict(X_scaled_dt)[0]
            traffic_count = self.lr_model.predict(X_scaled_lr)[0]
            
            # Ensure predictions are within reasonable bounds
            traffic_count = np.clip(traffic_count, 0, 2000)
            
            # Create figure with two subplots
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
            
            # Plot 1: Current predictions
            ax1.bar(['Traffic Volume'], [traffic_count], color='skyblue')
            ax1.set_title('Predicted Traffic Volume')
            ax1.set_ylabel('Vehicles per Hour')
            
            # Plot 2: 24-hour prediction with historical context
            hours = range(24)
            predictions = []
            historical_avg = []
            
            for h in hours:
                # Get predictions
                features.loc[0, 'hour_sin'] = np.sin(2 * np.pi * h/24)
                features.loc[0, 'hour_cos'] = np.cos(2 * np.pi * h/24)
                features.loc[0, 'is_morning_peak'] = 1 if 6 <= h <= 9 else 0
                features.loc[0, 'is_evening_peak'] = 1 if 16 <= h <= 19 else 0
                
                # Update features with typical values for that hour
                typical_values = self.get_typical_values(h, self.is_weekend_widget.value)
                features.loc[0, 'rolling_avg_3h'] = typical_values['rolling_avg']
                features.loc[0, 'traffic_density'] = typical_values['traffic_density']
                
                X_scaled = self.lr_scaler.transform(features)
                pred = self.lr_model.predict(X_scaled)[0]
                predictions.append(np.clip(pred, 0, 2000))
                
                # Get historical average for context
                mask = (self.historical_data['hour'] == h) & \
                       (self.historical_data['is_weekend'] == self.is_weekend_widget.value)
                hist_avg = self.historical_data[mask]['traffic_count'].mean()
                historical_avg.append(hist_avg)
            
            ax2.plot(hours, predictions, marker='o', label='Predicted')
            ax2.plot(hours, historical_avg, '--', label='Historical Average', alpha=0.5)
            ax2.set_title('24-Hour Traffic Prediction')
            ax2.set_xlabel('Hour of Day')
            ax2.set_ylabel('Predicted Traffic Volume')
            ax2.legend()
            
            plt.tight_layout()
            plt.show()
            
            # Display numeric predictions
            congestion_labels = ['Light', 'Moderate', 'High', 'Severe']
            print(f"\nPredictions for Hour {self.hour_widget.value}:00")
            print(f"Traffic Volume: {int(traffic_count)} vehicles/hour")
            print(f"Congestion Level: {congestion_labels[int(congestion_level)]}")
            
            # Show historical context
            mask = (self.historical_data['hour'] == self.hour_widget.value) & \
                   (self.historical_data['is_weekend'] == self.is_weekend_widget.value)
            hist_avg = self.historical_data[mask]['traffic_count'].mean()
            print(f"\nHistorical average for this hour: {int(hist_avg)} vehicles/hour")
    
    def display_dashboard(self):
        """Display the dashboard."""
        # Create layout
        input_widgets = widgets.VBox([
            widgets.HBox([self.hour_widget, self.is_weekend_widget]),
            widgets.HBox([self.temperature_widget, self.humidity_widget]),
            widgets.HBox([self.wind_speed_widget, self.visibility_widget]),
            widgets.HBox([self.precipitation_widget, self.weather_widget]),
            widgets.HBox([self.rolling_avg_widget, self.traffic_density_widget]),
            self.update_button
        ])
        
        # Display dashboard
        display(input_widgets)
        display(self.output)
        
        # Initial update
        self.update_predictions(None)

# Create and display dashboard
print("Initializing Traffic Prediction Dashboard...")
dashboard = TrafficDashboard()
dashboard.display_dashboard()

Initializing Traffic Prediction Dashboard...
Loading models and scalers...


Output()